In [10]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import matplotlib.dates as mdates
import datetime
import matplotlib.ticker as ticker

df=pd.read_csv("fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv")
df["Date"]=pd.to_datetime(df["Date"])
df["Year"]=df["Date"].dt.year
df["Month"]=df["Date"].dt.month
df["Month_Date"]=df["Date"].dt.strftime("%m-%d")
df=df[df["Month_Date"]!="02-29"]
df["Data_Value"]=df["Data_Value"]*0.1
month={1:"Jan",2:"Feb",3:"March",4:"April",5:"May",6:"June",7:"July",8:"Aug",9:"Sept",10:"Oct",11:"Nov",12:"Dec"}
df["monthName"]=df.Month.map(month)

maxt=df[df["Element"]=="TMAX"].reset_index(drop=True)
mint=df[df["Element"]=="TMIN"].reset_index(drop=True)

maxt=maxt.sort_values("Date").reset_index(drop=True)
mint=mint.sort_values("Date").reset_index(drop=True)

x=maxt.groupby(["Month_Date"])["Data_Value"].max()
y=mint.groupby("Month_Date")["Data_Value"].min()

cmax=maxt[maxt["Year"]==2015].reset_index(drop=True)
cmin=mint[mint["Year"]==2015].reset_index(drop=True)
xo=cmax.groupby("Date")["Data_Value"].max()
yo=cmin.groupby("Date")["Data_Value"].min()

zf=pd.DataFrame(x)
zf.reset_index(inplace=True)
zf["maxt"]=True
zf.set_index(["Month_Date","Data_Value"],inplace=True)
maxt.set_index(["Month_Date","Data_Value"],inplace=True)

xf=pd.merge(maxt,zf,how="inner",left_index=True,right_index=True)
xf.reset_index(inplace=True)
xf.drop_duplicates(subset="Month_Date",inplace=True)
xf.set_index("Month_Date",inplace=True)

wf=pd.DataFrame(y)
wf.reset_index(inplace=True)
wf["mint"]=True
wf.set_index(["Month_Date","Data_Value"],inplace=True)
mint.set_index(["Month_Date","Data_Value"],inplace=True)

yf=pd.merge(mint,wf,how="inner",left_index=True,right_index=True)
yf.reset_index(inplace=True)
yf.drop_duplicates(subset="Month_Date",inplace=True)
yf.set_index("Month_Date",inplace=True)

cx=xf.set_index("Date")
a=pd.merge(cx["Data_Value"],xo,how="inner",left_index=True,right_index=True,on=["Data_Value"])
dx=yf.set_index("Date")
b=pd.merge(dx["Data_Value"],yo,how="inner",left_index=True,right_index=True,on=["Data_Value"])

ds=(np.arange("2015-01-01","2016-01-01",dtype='datetime64[D]')).tolist()

plt.figure()
plt.plot(ds,xf["Data_Value"].values.tolist(),lw=1,c="green")
plt.plot(ds,yf["Data_Value"].values.tolist(),lw=1,c="lightcoral")
plt.scatter(a.index,a["Data_Value_x"],color="red",alpha=1,s=25)
plt.scatter(b.index,b["Data_Value_x"],color="lightgreen",alpha=1,s=25)
i=x.index.tolist()
plt.gca().fill_between(ds,
                       xf["Data_Value"].values.tolist(),yf["Data_Value"].values.tolist(),
                      color="grey",alpha=0.25)

                       
plt.legend(['Record high (2005-2014)','Record low (2005-2014)','Record breaking high in 2015','Record breaking low in 2015'],loc=0,frameon=False
          ,prop={"size":6})

plt.xlabel('Months', fontsize=10)
plt.ylabel('° Celsius', fontsize=10)
plt.title('Max vs Min temp.', fontsize=12)
ax = plt.gca()
monthformat = mdates.DateFormatter('%b')
months = mdates.MonthLocator()
ax.xaxis.set_major_locator(months)
ax.xaxis.set_major_formatter(monthformat)
ax.set_xlim([datetime.date(2015, 1, 1), datetime.date(2015, 12, 31)])
for i in plt.gca().spines.values():
    i.set_visible(False)
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=15))
for tick in ax.xaxis.get_minor_ticks():
    tick.tick1line.set_markersize(0) # Make small ticker disappear
    tick.label1.set_horizontalalignment('center')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …